# Práctica: Contrastes


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Cargar datos reales: Longley
longley = sm.datasets.longley.load_pandas().data.copy()
longley


,TOTEMP,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,60323.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,61122.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,60171.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,61187.0,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,63221.0,96.2,328975.0,2099.0,3099.0,112075.0,1951.0
5,63639.0,98.1,346999.0,1932.0,3594.0,113270.0,1952.0
6,64989.0,99.0,365385.0,1870.0,3547.0,115094.0,1953.0
7,63761.0,100.0,363112.0,3578.0,3350.0,116219.0,1954.0
8,66019.0,101.2,397469.0,2904.0,3048.0,117388.0,1955.0
9,67857.0,104.6,419180.0,2822.0,2857.0,118734.0,1956.0


### Variables del dataset Longley
- `TOTEMP`: Empleo total (variable dependiente).  
- `GNPDEFL`: Deflactor del GNP.  
- `GNP`: GNP (Producto Nacional Bruto).  
- `UNEMP`: Desempleo (en miles).  
- `ARMED`: Fuerzas Armadas (en miles).  
- `POP`: Población.  
- `YEAR`: Año.


In [ ]:
# Comprobar nombres exactos de columnas en su instalación
longley.columns


## 1) Ajuste del modelo OLS


In [4]:
y = longley["TOTEMP"]

X_cols = [c for c in longley.columns if c != "TOTEMP"]
X = longley[X_cols].astype(float)


# Ajustar OLS
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 TOTEMP   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     330.3
Date:                Tue, 04 Nov 2025   Prob (F-statistic):           4.98e-10
Time:                        07:49:58   Log-Likelihood:                -109.62
No. Observations:                  16   AIC:                             233.2
Df Residuals:                       9   BIC:                             238.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.482e+06    8.9e+05     -3.911      0.0

## 2) Significación **global**

La prueba F en el resumen contrasta:  
$$
H_0: \beta_1=\beta_2=\cdots=\beta_k=0
$$
frente a la alternativa de que **algún** coeficiente es no nulo.


In [5]:
print("Estadístico F global:", results.fvalue)
print("p-valor F global:", results.f_pvalue)

Estadístico F global: 330.28533923414756
p-valor F global: 4.984030528754532e-10


## 3) Significación **individual**

Para cada \( \beta_j \):  
$$
H_0: \beta_j = 0
$$


In [6]:
coef_table = pd.DataFrame({
    "coef": results.params,
    "std_err": results.bse,
    "t": results.tvalues,
    "p>|t|": results.pvalues
})
coef_table


,coef,std_err,t,p>|t|
const,-3.482259e+06,890420.383608,-3.910803,0.003560
GNPDEFL,1.506187e+01,84.914926,0.177376,0.863141
GNP,-3.581918e-02,0.033491,-1.069516,0.312681
UNEMP,-2.020230e+00,0.488400,-4.136427,0.002535
ARMED,-1.033227e+00,0.214274,-4.821985,0.000944
POP,-5.110411e-02,0.226073,-0.226051,0.826212
YEAR,1.829151e+03,455.478499,4.015890,0.003037


## 4) Contrastes sobre **relaciones lineales** \(R\beta = r\)

Además de probar si un coeficiente es 0, a menudo queremos saber si **dos efectos son iguales** o si **una combinación** de efectos cumple cierta relación.


In [7]:
# Ver el orden de parámetros (índices de beta)
results.params.index


Index(['const', 'GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP', 'YEAR'], dtype='object')

### 4.1) Igualdad de dos coeficientes: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}}$

Interpretación: “¿Un incremento marginal en GNP tiene el **mismo** efecto sobre el empleo que un incremento marginal en UNEMP (desempleo)?”


In [9]:

names = results.params.index.tolist()

# Construir restricción: beta_GNP - beta_UNEMP = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNP")]   = 1.0
R[0, names.index("UNEMP")] = -1.0
r = np.array([0.0])

ftest_eq = results.f_test((R, r))   # F-test de igualdad

print("== Igualdad de coeficientes: beta_GNP = beta_UNEMP ==")
print("F-stat:", ftest_eq.fvalue, "p-value:", ftest_eq.pvalue)
ftest_eq


== Igualdad de coeficientes: beta_GNP = beta_UNEMP ==
F-stat: 18.86671600551824 p-value: 0.0018676657918953873


<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=18.86671600551824, p=0.0018676657918953873, df_denom=9, df_num=1>

### 4.2) Grupo de coeficientes nulos: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}} = 0$


In [10]:
R = np.zeros((2, len(names)))
r = np.zeros(2)

R[0, names.index("GNP")]   = 1.0           # beta_GNP = 0
R[1, names.index("UNEMP")] = 1.0           # beta_UNEMP = 0

ftest_group_zero = results.f_test((R, r))
print("== Grupo cero: beta_GNP = beta_UNEMP = 0 ==")
print("F-stat:", float(ftest_group_zero.fvalue), "p-value:", float(ftest_group_zero.pvalue))


== Grupo cero: beta_GNP = beta_UNEMP = 0 ==
F-stat: 46.7141384089514 p-value: 1.766859000548018e-05


### 4.3) Combinación lineal: $\beta_{\text{GNPDEFL}} + \beta_{\text{POP}} = \beta_{\text{ARMED}}$



In [11]:

# Restricción: beta_GNPDEFL + beta_POP - beta_ARMED = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNPDEFL")] = 1.0
R[0, names.index("POP")]  = 1.0
R[0, names.index("ARMED")]  = -1.0
r = np.array([0.0])

ftest_combo = results.f_test((R, r))
print("== Combinación: beta_GNPDEFL + beta_POP = beta_ARMED ==")
print("F-stat:", float(ftest_combo.fvalue), "p-value:", float(ftest_combo.pvalue))


== Combinación: beta_GNPDEFL + beta_POP = beta_ARMED ==
F-stat: 0.03551135120467506 p-value: 0.8547088211410531


# Cuantiles Tablas

In [12]:
from scipy.stats import t, f, chi2

In [14]:
t.ppf(1-0.025, 16 - 7)

np.float64(2.2621571628540993)

In [16]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 TOTEMP   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     330.3
Date:                Tue, 04 Nov 2025   Prob (F-statistic):           4.98e-10
Time:                        08:47:03   Log-Likelihood:                -109.62
No. Observations:                  16   AIC:                             233.2
Df Residuals:                       9   BIC:                             238.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.482e+06    8.9e+05     -3.911      0.0

In [17]:
f.ppf(1-0.05, 6, 16-7)

np.float64(3.373753647039213)

In [19]:
sigma_gorro=results.scale

In [20]:
(16-7)*sigma_gorro/chi2.ppf(1-0.025, 16-7)

np.float64(43969.6296753805)

In [21]:
(16-7)*sigma_gorro/chi2.ppf(0.025, 16-7)

np.float64(309742.00407500827)

In [22]:
sigma_gorro

np.float64(92936.0061674473)

In [23]:
np.sqrt(sigma_gorro)

np.float64(304.8540735621673)